# Brownian dynamics simulator

In [ ]:
import numpy as np
from matplotlib.pyplot import *
%matplotlib notebook
from scipy import *
from tqdm.notebook import *
from matplotlib.animation import *

# Rk4 integrator and functions definitions

In [ ]:
#########################
#numerical integrator####


def integrate_euler(F,t,y,tstop,dt):

    def euler(F,t,y,dt):
        #retorna o incremento apra utilizar na resolucao
        K0=dt*F(t,y)
        return K0

    T=[]
    Y=[]
    T.append(t)
    Y.append(y)
    
    pbar = tqdm(total=(tstop-dt))
    while t<tstop:
        pbar.update(dt)
        dt=min(dt,tstop-t)
        y=y+euler(F,t,y,dt)
        t=t+dt
        T.append(t)
        Y.append(y)

    return np.array(T),np.array(Y) 


##################################
#functions for simulating the dynamics

def harmonic_trap_potential(x,y,z,k,theta):
    kx=k[0]
    ky=k[1]
    kz=k[2]
    return - 1/2 *((kx*(x*np.cos(theta)-y*np.sin(theta))**2)+
                     (ky*(x*np.sin(theta)+y*np.cos(theta))**2) + 1/2*kz*z**2)

def harmonic_trap(x,y,z,k,theta):

    
    kx=k[0]
    ky=k[1]
    kz=k[2]
    
    Fx = - (kx*(x*np.cos(theta)-y*np.sin(theta))*np.cos(theta)+(ky*(x*np.sin(theta)+y*np.cos(theta))*np.sin(theta)))
    Fy = - (-kx*(x*np.cos(theta)-y*np.sin(theta))*np.sin(theta)+(ky*(x*np.sin(theta)+y*np.cos(theta))*np.cos(theta)))
    Fz = - kz*z
    return Fx,Fy,Fz

def brownian_simulation(t_stop, dt, k, W, gamma, theta = 0, m=1,delta_x=0,delta_v=0):
    print('k',k)
    print('theta',theta)
    print('W',W)
    def F(t,y):

        Fi=np.zeros(6) #[x,vx,y,vy,z,vz]
        Fx,Fy,Fz = harmonic_trap(y[0],y[2],y[4],k,theta)
        
        #normalization factor of the noise - Variance of the noise must be equal to 1/dt
        W1 = W/np.sqrt(dt)
        
        Fi[0]=y[1] 
        Fi[1]= -(gamma/m) * y[1] + Fx/m   + W1/m*(np.random.normal(scale=1))
        
        Fi[2]=y[3]  
        Fi[3]= -(gamma/m) * y[3] + Fy/m  + W1/m*(np.random.normal(scale=1))
        
        Fi[4]=y[5] 
        Fi[5]= -(gamma/m) * y[5] + Fz/m + W1/m*(np.random.normal(scale=1))
            
        return Fi

    
    def initial_condition():
        y=np.zeros(6)

        for i in range(0,6,2):

            y[i] = delta_x*(1-2*np.random.rand())
            y[i+1] = delta_v*(1-2*np.random.rand())

        #define initial position
        y[0]= 10
        y[2]= 10
        y[4]= 10
        
        return y
    
    r0=initial_condition()
    T,Y=integrate_euler(F,0,r0,t_stop,dt)
    return T,Y


# Simulation of the Brownian Motion

In [ ]:
#[kx,ky,kz] - parameters of the harmonic trap
k_0 = [.9,.2,.5]

#noise parameters
#drag parameter
m=1
gamma_0 = .2
kB = 1
T0 = 1
D = kB * T0 / gamma_0
W_0 = np.sqrt(2*kB*T0*gamma_0)


#xy angle
theta = 50


#simulation time, dt step
t_stop = 10000
dt = 0.01

#simulation Y is a vector with [x,vx,y,vy,z,vz](t)
initial_pos = []
T,Y = brownian_simulation(t_stop,dt, k_0, W_0, gamma = gamma_0, m = m , theta = theta, delta_x=2)

time = T
x_data = Y[:,0]
y_data = Y[:,2] 
z_data = Y[:,4]

In [ ]:
k_0

# Parameters of the trap

In [ ]:
x = np.arange(-3,3.1,0.1) 
y = np.arange(-3,3.1,0.1)
xx,yy = np.meshgrid(x,y)

Fx,Fy,Vz = harmonic_trap(xx,yy,0,k_0,theta)
V = harmonic_trap_potential(xx,yy,0,k_0,theta)


fig = figure (figsize=[8,12])

G = GridSpec(5,3,  width_ratios=[1,1,1], height_ratios=[0.3, 0.3, 0.3,1,1])

ax = fig.add_subplot(G[0,2:])
ax.plot(time, x_data,lw=0.4,color='k')
ax.set_title(r'$x(t)$',fontsize=16)
ax.set_xlim([time.min(),time.max()])
ax.set_xlabel(r'$t(arb.un.)$',fontsize=16)
ax.set_xticks([time.min(),time.max()])
ax.set_xticklabels([int(time.min()),int(time.max())],fontsize=14)

ax.set_ylabel(r'$x(arb.un.)$',fontsize=16)
ax.set_ylim([x_data.min(),x_data.max()])
ax.set_yticks([x_data.min(),0,x_data.max()])
ax.set_yticklabels([int(y.min()),0,int(y.max())],fontsize=14)

ax = fig.add_subplot(G[1,2:])
ax.plot(time, y_data,lw=0.4,color='b')
ax.set_title(r'$y(t)$',fontsize=16)
ax.set_xlim([time.min(),time.max()])
ax.set_xlabel(r'$t(arb.un.)$',fontsize=16)
ax.set_xticks([time.min(),0,time.max()])
ax.set_xticklabels([int(time.min()),0,int(time.max())],fontsize=14)

ax.set_ylabel(r'$y(arb.un.)$',fontsize=16)
ax.set_ylim([y_data.min(),y_data.max()])
ax.set_yticks([y_data.min(),0,y_data.max()])
ax.set_yticklabels([int(y_data.min()),0,int(y_data.max())],fontsize=14)

ax = fig.add_subplot(G[2,2:])
ax.plot(time, z_data,lw=0.4,color='g')
ax.set_title(r'$z(t)$',fontsize=16)
ax.set_xlim([time.min(),time.max()])
ax.set_xlabel(r'$t(arb.un.)$',fontsize=16)
ax.set_xticks([time.min(),0,time.max()])
ax.set_xticklabels([int(time.min()),0,int(time.max())],fontsize=14)

ax.set_ylabel(r'$z(arb.un.)$',fontsize=16)
ax.set_ylim([z_data.min(),z_data.max()])
ax.set_yticks([z_data.min(),0,z_data.max()])
ax.set_yticklabels([int(z_data.min()),0,int(z_data.max())],fontsize=14)

##############################################################

ax = fig.add_subplot(G[3,0])
ax.imshow(V,extent=[np.min(x),np.max(x),np.min(y),np.max(y)],origin='lower',alpha=0.6,cmap=cm.viridis_r)
CS = ax.contour(V,levels=[np.min(V),-1.5,-1.0,-.5,np.max(V)],extent=[np.min(x),np.max(x),np.min(y),np.max(y)],cmap=cm.viridis_r)
ax.clabel(CS, CS.levels, inline=True, fontsize=12)
ax.streamplot(xx,yy,Fx,Fy, density=[.5,.5],color='k', linewidth=.3)
ax.set_title(r'$V(x,y,z=0)$',fontsize=16)
ax.set_xlim([x.min(),x.max()])
ax.set_xlabel(r'$x$',fontsize=16)
ax.set_xticks([x.min(),0,x.max()])
ax.set_xticklabels([int(x.min()),0,int(x.max())],fontsize=14)

ax.set_ylabel(r'$y$',fontsize=16)
ax.set_ylim([y.min(),y.max()])
ax.set_yticks([y.min(),0,y.max()])
ax.set_yticklabels([int(y.min()),0,int(y.max())],fontsize=14)

##############################
x = np.arange(-3,3.1,0.1) 
y = np.arange(-3,3.1,0.1)
z = np.arange(-3,3.1,0.1)
xx,zz = np.meshgrid(x,z)

Fx,Fy,Fz = harmonic_trap(xx,0,zz,k_0,theta)
V = harmonic_trap_potential(xx,0,zz,k_0,theta)

ax1 = fig.add_subplot(G[3,1])
ax1.imshow(V,extent=[np.min(x),np.max(x),np.min(y),np.max(y)],origin='lower',alpha=0.6,cmap=cm.viridis_r)
CS = ax1.contour(V,levels=[np.min(V),-1.5,-1.0,-.5,np.max(V)],extent=[np.min(x),np.max(x),np.min(y),np.max(y)],cmap=cm.viridis_r)
ax1.clabel(CS, CS.levels, inline=True, fontsize=12)
ax1.streamplot(xx,yy,Fx,Fz, density=[.5,.5],color='k', linewidth=.3)
ax1.set_title(r'$V(x,y=0,z)$',fontsize=16)
ax1.set_xlim([x.min(),x.max()])
ax1.set_xlabel(r'$x$',fontsize=16)
ax1.set_xticks([x.min(),0,x.max()])
ax1.set_xticklabels([int(x.min()),0,int(x.max())],fontsize=14)

ax1.set_ylabel(r'$z$',fontsize=16)
ax1.set_ylim([z.min(),z.max()])
ax1.set_yticks([z.min(),0,z.max()])
ax1.set_yticklabels([int(z.min()),0,int(z.max())],fontsize=14)

##############################
x = np.arange(-3,3.1,0.1) 
y = np.arange(-3,3.1,0.1)
z = np.arange(-3,3.1,0.1)
yy,zz = np.meshgrid(x,z)

Fx,Fy,Fz = harmonic_trap(0,yy,zz,k_0,theta)
V = harmonic_trap_potential(0,yy,zz,k_0,theta)

ax2 = fig.add_subplot(G[3,2])
ax2.imshow(V,extent=[np.min(x),np.max(x),np.min(y),np.max(y)],origin='lower',alpha=0.6,cmap=cm.viridis_r)
CS = ax2.contour(V,levels=[np.min(V),-1.5,-1.0,-.5,np.max(V)],extent=[np.min(x),np.max(x),np.min(y),np.max(y)],cmap=cm.viridis_r)
ax2.clabel(CS, CS.levels, inline=True, fontsize=12)
ax2.streamplot(yy,zz,Fy,Fz, density=[.5,.5],color='k', linewidth=.3)
ax2.set_title(r'$V(x=0,y,z)$',fontsize=16)
ax2.set_xlim([y.min(),y.max()])
ax2.set_xlabel(r'$y$',fontsize=16)
ax2.set_xticks([y.min(),0,y.max()])
ax2.set_xticklabels([int(y.min()),0,int(y.max())],fontsize=14)

ax2.set_ylabel(r'$z$',fontsize=16)
ax2.set_ylim([z.min(),z.max()])
ax2.set_yticks([z.min(),0,z.max()])
ax2.set_yticklabels([int(z.min()),0,int(z.max())],fontsize=14)


ax2 = fig.add_subplot(G[4,0])
bins = ax2.hexbin(x_data, y_data, extent = [x.min(),x.max(),y.min(),y.max()], gridsize=20,cmap=cm.inferno_r,)
ax2.set_title(r'$X vs Y$',fontsize=16)
ax2.set_xlim([x.min(),x.max()])
ax2.set_xlabel(r'$x$',fontsize=16)
ax2.set_xticks([x.min(),0,x.max()])
ax2.set_xticklabels([int(x.min()),0,int(x.max())],fontsize=14)

ax2.set_ylabel(r'$y$',fontsize=16)
ax2.set_ylim([y.min(),y.max()])
ax2.set_yticks([y.min(),0,y.max()])
ax2.set_yticklabels([int(y.min()),0,int(y.max())],fontsize=14)

ax2 = fig.add_subplot(G[4,1])
bins = ax2.hexbin(x_data, z_data, extent = [x.min(),x.max(),z.min(),z.max()], gridsize=20,cmap=cm.inferno_r,)
ax2.set_title(r'$X vs Z$',fontsize=16)
ax2.set_xlim([x.min(),x.max()])
ax2.set_xlabel(r'$x$',fontsize=16)
ax2.set_xticks([x.min(),0,x.max()])
ax2.set_xticklabels([int(x.min()),0,int(x.max())],fontsize=14)

ax2.set_ylabel(r'$z$',fontsize=16)
ax2.set_ylim([z.min(),z.max()])
ax2.set_yticks([z.min(),0,z.max()])
ax2.set_yticklabels([int(z.min()),0,int(z.max())],fontsize=14)


ax2 = fig.add_subplot(G[4,2])
bins = ax2.hexbin(y_data, z_data, extent = [y.min(),y.max(),z.min(),z.max()], gridsize=20,cmap=cm.inferno_r,)
ax2.set_title(r'$Y vs Z$',fontsize=16)
ax2.set_xlim([y.min(),y.max()])
ax2.set_xlabel(r'$y$',fontsize=16)
ax2.set_xticks([y.min(),0,y.max()])
ax2.set_xticklabels([int(y.min()),0,int(y.max())],fontsize=14)

ax2.set_ylabel(r'$z$',fontsize=16)
ax2.set_ylim([z.min(),z.max()])
ax2.set_yticks([z.min(),0,z.max()])
ax2.set_yticklabels([int(z.min()),0,int(z.max())],fontsize=14)



ax = fig.add_subplot(G[:3,:2],projection='3d')
line = ax.plot(x_data,y_data,z_data, lw=0.5, c='b',alpha=0.5) 
ax.view_init(60, 35)

fig.tight_layout()

In [ ]:
x = np.arange(-3,3.1,0.1) 
y = np.arange(-3,3.1,0.1)
xx,yy = np.meshgrid(x,y)

Fx,Fy,Vz = harmonic_trap(xx,yy,0,k_0,theta)
V = harmonic_trap_potential(xx,yy,0,k_0,theta)


fig = figure (figsize=[8,12])

G = GridSpec(5,3,  width_ratios=[1,1,1], height_ratios=[0.3, 0.3, 0.3,1,1])

ax = fig.add_subplot(G[0,2:])
ax.plot(T, x_data,lw=0.4,color='k')
ax.set_title(r'$x(t)$',fontsize=16)
ax.set_xlim([T.min(),T.max()])
ax.set_xlabel(r'$t(arb.un.)$',fontsize=16)
ax.set_xticks([T.min(),T.max()])
ax.set_xticklabels([int(T.min()),int(T.max())],fontsize=14)

ax.set_ylabel(r'$x(arb.un.)$',fontsize=16)
ax.set_ylim([x_data.min(),x_data.max()])
ax.set_yticks([x_data.min(),0,x_data.max()])
ax.set_yticklabels([int(y.min()),0,int(y.max())],fontsize=14)

ax = fig.add_subplot(G[1,2:])
ax.plot(T, y_data,lw=0.4,color='b')
ax.set_title(r'$y(t)$',fontsize=16)
ax.set_xlim([T.min(),T.max()])
ax.set_xlabel(r'$t(arb.un.)$',fontsize=16)
ax.set_xticks([T.min(),0,T.max()])
ax.set_xticklabels([int(T.min()),0,int(T.max())],fontsize=14)

ax.set_ylabel(r'$y(arb.un.)$',fontsize=16)
ax.set_ylim([y_data.min(),y_data.max()])
ax.set_yticks([y_data.min(),0,y_data.max()])
ax.set_yticklabels([int(y_data.min()),0,int(y_data.max())],fontsize=14)

ax = fig.add_subplot(G[2,2:])
ax.plot(T, z_data,lw=0.4,color='g')
ax.set_title(r'$z(t)$',fontsize=16)
ax.set_xlim([T.min(),T.max()])
ax.set_xlabel(r'$t(arb.un.)$',fontsize=16)
ax.set_xticks([T.min(),0,T.max()])
ax.set_xticklabels([int(T.min()),0,int(T.max())],fontsize=14)

ax.set_ylabel(r'$z(arb.un.)$',fontsize=16)
ax.set_ylim([z_data.min(),z_data.max()])
ax.set_yticks([z_data.min(),0,z_data.max()])
ax.set_yticklabels([int(z_data.min()),0,int(z_data.max())],fontsize=14)

##############################################################

ax = fig.add_subplot(G[3,0])
ax.imshow(V,extent=[np.min(x),np.max(x),np.min(y),np.max(y)],origin='lower',alpha=0.6,cmap=cm.viridis_r)
CS = ax.contour(V,levels=[np.min(V),-1.5,-1.0,-.5,np.max(V)],extent=[np.min(x),np.max(x),np.min(y),np.max(y)],cmap=cm.viridis_r)
ax.clabel(CS, CS.levels, inline=True, fontsize=12)
ax.streamplot(xx,yy,Fx,Fy, density=[.5,.5],color='k', linewidth=.3)
ax.set_title(r'$V(x,y,z=0)$',fontsize=16)
ax.set_xlim([x.min(),x.max()])
ax.set_xlabel(r'$x$',fontsize=16)
ax.set_xticks([x.min(),0,x.max()])
ax.set_xticklabels([int(x.min()),0,int(x.max())],fontsize=14)

ax.set_ylabel(r'$y$',fontsize=16)
ax.set_ylim([y.min(),y.max()])
ax.set_yticks([y.min(),0,y.max()])
ax.set_yticklabels([int(y.min()),0,int(y.max())],fontsize=14)

##############################
x = np.arange(-3,3.1,0.1) 
y = np.arange(-3,3.1,0.1)
z = np.arange(-3,3.1,0.1)
xx,zz = np.meshgrid(x,z)

Fx,Fy,Fz = harmonic_trap(xx,0,zz,k_0,theta)
V = harmonic_trap_potential(xx,0,zz,k_0,theta)

ax1 = fig.add_subplot(G[3,1])
ax1.imshow(V,extent=[np.min(x),np.max(x),np.min(y),np.max(y)],origin='lower',alpha=0.6,cmap=cm.viridis_r)
CS = ax1.contour(V,levels=[np.min(V),-1.5,-1.0,-.5,np.max(V)],extent=[np.min(x),np.max(x),np.min(y),np.max(y)],cmap=cm.viridis_r)
ax1.clabel(CS, CS.levels, inline=True, fontsize=12)
ax1.streamplot(xx,yy,Fx,Fz, density=[.5,.5],color='k', linewidth=.3)
ax1.set_title(r'$V(x,y=0,z)$',fontsize=16)
ax1.set_xlim([x.min(),x.max()])
ax1.set_xlabel(r'$x$',fontsize=16)
ax1.set_xticks([x.min(),0,x.max()])
ax1.set_xticklabels([int(x.min()),0,int(x.max())],fontsize=14)

ax1.set_ylabel(r'$z$',fontsize=16)
ax1.set_ylim([z.min(),z.max()])
ax1.set_yticks([z.min(),0,z.max()])
ax1.set_yticklabels([int(z.min()),0,int(z.max())],fontsize=14)

##############################
x = np.arange(-3,3.1,0.1) 
y = np.arange(-3,3.1,0.1)
z = np.arange(-3,3.1,0.1)
yy,zz = np.meshgrid(x,z)

Fx,Fy,Fz = harmonic_trap(0,yy,zz,k_0,theta)
V = harmonic_trap_potential(0,yy,zz,k_0,theta)

ax2 = fig.add_subplot(G[3,2])
ax2.imshow(V,extent=[np.min(x),np.max(x),np.min(y),np.max(y)],origin='lower',alpha=0.6,cmap=cm.viridis_r)
CS = ax2.contour(V,levels=[np.min(V),-1.5,-1.0,-.5,np.max(V)],extent=[np.min(x),np.max(x),np.min(y),np.max(y)],cmap=cm.viridis_r)
ax2.clabel(CS, CS.levels, inline=True, fontsize=12)
ax2.streamplot(yy,zz,Fy,Fz, density=[.5,.5],color='k', linewidth=.3)
ax2.set_title(r'$V(x=0,y,z)$',fontsize=16)
ax2.set_xlim([y.min(),y.max()])
ax2.set_xlabel(r'$y$',fontsize=16)
ax2.set_xticks([y.min(),0,y.max()])
ax2.set_xticklabels([int(y.min()),0,int(y.max())],fontsize=14)

ax2.set_ylabel(r'$z$',fontsize=16)
ax2.set_ylim([z.min(),z.max()])
ax2.set_yticks([z.min(),0,z.max()])
ax2.set_yticklabels([int(z.min()),0,int(z.max())],fontsize=14)


ax2 = fig.add_subplot(G[4,0])
bins = ax2.hexbin(x_data, y_data, extent = [x.min(),x.max(),y.min(),y.max()], gridsize=20,cmap=cm.inferno_r,)
ax2.set_title(r'$X vs Y$',fontsize=16)
ax2.set_xlim([x.min(),x.max()])
ax2.set_xlabel(r'$x$',fontsize=16)
ax2.set_xticks([x.min(),0,x.max()])
ax2.set_xticklabels([int(x.min()),0,int(x.max())],fontsize=14)

ax2.set_ylabel(r'$y$',fontsize=16)
ax2.set_ylim([y.min(),y.max()])
ax2.set_yticks([y.min(),0,y.max()])
ax2.set_yticklabels([int(y.min()),0,int(y.max())],fontsize=14)

ax2 = fig.add_subplot(G[4,1])
bins = ax2.hexbin(x_data, z_data, extent = [x.min(),x.max(),z.min(),z.max()], gridsize=20,cmap=cm.inferno_r,)
ax2.set_title(r'$X vs Z$',fontsize=16)
ax2.set_xlim([x.min(),x.max()])
ax2.set_xlabel(r'$x$',fontsize=16)
ax2.set_xticks([x.min(),0,x.max()])
ax2.set_xticklabels([int(x.min()),0,int(x.max())],fontsize=14)

ax2.set_ylabel(r'$z$',fontsize=16)
ax2.set_ylim([z.min(),z.max()])
ax2.set_yticks([z.min(),0,z.max()])
ax2.set_yticklabels([int(z.min()),0,int(z.max())],fontsize=14)


ax2 = fig.add_subplot(G[4,2])
bins = ax2.hexbin(y_data, z_data, extent = [y.min(),y.max(),z.min(),z.max()], gridsize=20,cmap=cm.inferno_r,)
ax2.set_title(r'$Y vs Z$',fontsize=16)
ax2.set_xlim([y.min(),y.max()])
ax2.set_xlabel(r'$y$',fontsize=16)
ax2.set_xticks([y.min(),0,y.max()])
ax2.set_xticklabels([int(y.min()),0,int(y.max())],fontsize=14)

ax2.set_ylabel(r'$z$',fontsize=16)
ax2.set_ylim([z.min(),z.max()])
ax2.set_yticks([z.min(),0,z.max()])
ax2.set_yticklabels([int(z.min()),0,int(z.max())],fontsize=14)



ax = fig.add_subplot(G[:3,:2],projection='3d')
line, = ax.plot(x_data[0],y_data[0],z_data[0], lw=0.5, c='b',alpha=0.5)
redDot, = ax.plot(x_data[0],y_data[0],z_data[0],ms=8,color='r')
ax.view_init(60, 35)
ax.set_xlabel('X(t)')
ax.set_ylabel('Y(t)')
ax.set_zlabel('Z(t)')
ax.set_title('Trajectory of the Brownian particle')
ax.set_xlim((-10,10))
ax.set_ylim((-10,10))
ax.set_zlim((-10,10))


fig.tight_layout()

def func_anim(num):
    step = 10
    # NOTE: there is no .set_data() for 3 dim data...
    line.set_data(x_data[0:num*step][::step],y_data[0:num*step][::step])    
    line.set_3d_properties(z_data[0:num*step][::step])
    redDot.set_data(x_data[num*step][::step],y_data[num*step][::step])    
    redDot.set_3d_properties(z_data[num*step][::step])  
 
FuncAnimation(fig, func_anim, interval=1, blit=False)


In [ ]:
func_anim(1)

# Plot the Signals Obtained

In [ ]:
subplots()
title('Signals')
subplot(311)
plot(time, x_data,lw=0.1)
subplot(312)
plot(time, y_data,lw=0.1)
subplot(313)
plot(time, z_data,lw=0.1)

# Fit a damped oscillator model for the initial dynamics in x, y and z

# Compute the stiffness from the last part of the dynamics

# Compare the results with the predicted